In [1]:
import sqlite3

# Step 1: Create a SQLite Database and Tables
import sqlite3

def setup_database():
    connection = sqlite3.connect('doctor_appointments.db')
    cursor = connection.cursor()

    # Create doctors table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS doctors (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL
    )
    ''')

    # Create available_slots table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS available_slots (
        id INTEGER PRIMARY KEY,
        doctor_id INTEGER NOT NULL,
        slot_time TEXT NOT NULL,
        FOREIGN KEY (doctor_id) REFERENCES doctors (id)
    )
    ''')

    # Create appointments table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS appointments (
        id INTEGER PRIMARY KEY,
        doctor_id INTEGER NOT NULL,
        patient_name TEXT NOT NULL,
        appointment_time TEXT NOT NULL,
        FOREIGN KEY (doctor_id) REFERENCES doctors (id)
    )
    ''')

    connection.commit()
    connection.close()

setup_database()

In [2]:
# Step 2: Insert Dummy Doctor Data
def insert_dummy_data():
    connection = sqlite3.connect('doctor_appointments.db')
    cursor = connection.cursor()

    doctors = [
        (1, 'Dr. NS Dhar'),
        (2, 'Dr. John Doe'),
        (3, 'Dr. Jane Smith')
    ]

    cursor.executemany('INSERT INTO doctors (id, name) VALUES (?, ?)', doctors)

    available_slots = [
        (1, 1, '2024-07-15 10:00'),
        (2, 1, '2024-07-15 11:00'),
        (3, 1, '2024-07-15 14:00'),
        (4, 2, '2024-07-15 09:00'),
        (5, 2, '2024-07-15 13:00'),
        (6, 3, '2024-07-15 15:00')
    ]

    cursor.executemany('INSERT INTO available_slots (id, doctor_id, slot_time) VALUES (?, ?, ?)', available_slots)

    connection.commit()
    connection.close()

# Step 3: Fetch Available Slots for a Given Doctor
def get_doctor_id_by_name(doctor_name):
    connection = sqlite3.connect('doctor_appointments.db')
    cursor = connection.cursor()
    query = "SELECT id FROM doctors WHERE name = ?"
    cursor.execute(query, (doctor_name,))
    result = cursor.fetchone()
    connection.close()
    return result[0] if result else None

def get_available_slots(doctor_name):
    doctor_id = get_doctor_id_by_name(doctor_name)
    if doctor_id is None:
        return {"error": f"No doctor found with name {doctor_name}"}

    connection = sqlite3.connect('doctor_appointments.db')
    cursor = connection.cursor()
    query = "SELECT slot_time FROM available_slots WHERE doctor_id = ?"
    cursor.execute(query, (doctor_id,))
    slots = cursor.fetchall()
    connection.close()
    return [slot[0] for slot in slots]


insert_dummy_data()

# Example usage
doctor_name = "Dr. NS Dhar"
available_slots = get_available_slots(doctor_name)

if 'error' in available_slots:
    print(available_slots['error'])
else:
    print(f"Available slots for {doctor_name}:")
    for slot in available_slots:
        print(slot)

Available slots for Dr. NS Dhar:
2024-07-15 10:00
2024-07-15 11:00
2024-07-15 14:00
